In [3]:
!pip install pinecone-haystack


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 3.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.4/305.4 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: pinecone-client
    Found existing installation: pinecone-client 3.0.0
    Uninstalling pinecone-client-3.0.0:
      Successfully uninstalled pinecone-client-3.0.0


In [1]:
from haystack_integrations.document_stores.pinecone import PineconeDocumentStore


document_store = PineconeDocumentStore(api_key='9508d6bc-49e8-4514-9efe-55bb68d1c4d9', index='transformer', environment='gcp-starter', return_embedding=True)


In [2]:
from datasets import load_dataset

subjqa = load_dataset("subjqa", 'electronics')

In [3]:
import pandas as pd

dfs = {split: dset.to_pandas() for split, dset in subjqa.flatten().items()}

for split, df in dfs.items():
    print(f"Number of questions in {split}: {df['id'].nunique()}")

Number of questions in train: 1295
Number of questions in test: 358
Number of questions in validation: 255


In [4]:
from haystack import Document
documents = []
for split, df in dfs.items():
    # Exclude duplicate reviews
    docs = [Document(content=row["context"], meta={"item_id": row["title"], "question_id": row["id"], "split": split})
        for _,row in df.drop_duplicates(subset="context").iterrows()]
    documents.extend(docs)


In [5]:
from haystack.components.embedders import SentenceTransformersTextEmbedder, SentenceTransformersDocumentEmbedder
from haystack_integrations.components.retrievers.pinecone import PineconeDenseRetriever

In [18]:
!pip install "sentence-transformers>=2.2.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Using cached torch-2.1.2-cp311-cp311-manylinux1_x86_64.whl.metadata (25 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  

In [6]:
document_embedder = SentenceTransformersDocumentEmbedder()  
document_embedder.warm_up()
documents_with_embeddings = document_embedder.run(documents)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/59 [00:00<?, ?it/s]

In [9]:

document_store.write_documents(documents_with_embeddings.get("documents"))

Upserted vectors:   0%|          | 0/1875 [00:00<?, ?it/s]

1875

In [16]:
from haystack import Pipeline
from haystack.components.readers import ExtractiveReader

reader = ExtractiveReader()
reader.warm_up()

query_pipeline = Pipeline()
query_pipeline.add_component("text_embedder", SentenceTransformersTextEmbedder())
query_pipeline.add_component("retriever", PineconeDenseRetriever(document_store=document_store))
query_pipeline.add_component(instance=reader, name="reader")
query_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
query_pipeline.connect("retriever.documents", "reader.documents")

query = "How many languages are there?"

result = query_pipeline.run({"text_embedder": {"text": query}, "reader": {"query": query, "top_k": 2}})

print(result)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'reader': {'answers': [ExtractedAnswer(query='How many languages are there?', score=0.4423697590827942, data='chinese not english', document=Document(id=7634a0e9c6eb9936712972c968f12d6e4c216199f877c185071c08cc4b5f0ea2, content: 'this mouse is pretty good i ordered it for a good price it shipped fast and got it in a few daysa fe...', meta: {'item_id': 'B005CPGHAA', 'question_id': '8f75c2599808a17be3fe53ddcd68ad48', 'split': 'test'}, score: 0.126256302, embedding: vector of size 768), context=None, document_offset=ExtractedAnswer.Span(start=417, end=436), context_offset=None, meta={}), ExtractedAnswer(query='How many languages are there?', score=0.36799100041389465, data='two', document=Document(id=87b3f05ccdfe65864be7a24e68602fa49ffb8076ab9bb758ad7f6a5ffb27766f, content: 'I had high expectations when I purchased this item, and it met or exceeded them in every way.  I typ...', meta: {'item_id': 'B003VAGXWK', 'question_id': '065ab2a6beae6a12948777a3eb65caf4', 'split': 'train'}, score: 0.